In [42]:
from mip import *
import numpy as np
from tqdm import tqdm
import time

N = 500
alpha = 1

J = np.random.randint(-100,100,size=(N*alpha, N*alpha))/(10**6)
J = (J + J.T)/2
J = J - np.diag(J)
h = np.random.randint(-100,100,size=N*alpha)/(10**4)

In [43]:
with open('test_J.npy', 'wb') as f:
    np.save(f, J/4)

with open('test_h.npy', 'wb') as f:
    np.save(f, h/2 - sum([J[:,i]/8 for i in range(N*alpha)]) - sum([J[i,:]/8 for i in range(N*alpha)]))

In [44]:
time_1 = time.time()

model = Model("Ising", solver_name=GRB)
u = [model.add_var(var_type=BINARY) for _ in range(N*alpha)]
z = [[model.add_var(var_type=BINARY) for _ in range(N*alpha)] for _ in range(N*alpha)]

for i in tqdm(range(N*alpha)):
    for j in range(N*alpha):
        model += z[i][j] <= u[i]
        model += z[i][j] <= u[j]
        model += z[i][j] >= u[i] + u[j] -1

model.objective = minimize((-1/2)*quicksum(quicksum(J[i,j]*z[i][j] for i in range(N*alpha)) for j in range(N*alpha)) + quicksum(h[i]*u[i] for i in range(N*alpha)))
model.optimize(max_seconds=300)

processing_time = time.time() - time_1

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


100%|██████████| 500/500 [00:07<00:00, 68.77it/s]


Set parameter TimeLimit to value 300
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 750000 rows, 250500 columns and 1749500 nonzeros
Model fingerprint: 0xcdf9d2ac
Variable types: 0 continuous, 250500 integer (250500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-22, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0016000
Presolve removed 1500 rows and 500 columns (presolve time = 5s) ...
Presolve removed 374040 rows and 500 columns
Presolve time: 7.55s
Presolved: 375960 rows, 250000 columns, 884414 nonzeros
Variable types: 0 continuous, 250000 integer (250000 binary)
Found heuristic solution: objective -1

In [45]:
with open('test_J.npy', 'rb') as f:
    test_J = np.load(f)

with open('test_h.npy', 'rb') as f:
    test_h = np.expand_dims(np.load(f),axis=1)

if model.num_solutions:
    print("Processing time: ", processing_time)
    
    spins = []
    for i in range(N*alpha):
        if u[i].x >= 0.99:
            spins.append(1)
        else:
            spins.append(-1)

    print("Spins: ", spins)
    print("Objective function: ", (-1/2)*sum([sum([test_J[i,j]*spins[j] for j in range(N*alpha)])*spins[i] for i in range(N*alpha)]) + sum([test_h[i]*spins[i] for i in range(N*alpha)])[0])

Processing time:  187.24528694152832
Spins:  [1, 1, 1, 1, 1, 1, -1, 1, -1, 1, 1, 1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, 1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, -1, -1, 1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, 1, 1, 1, 1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1,

In [50]:
time_1 = time.time()

model = Model("Ising", solver_name=GRB)
u = [model.add_var(var_type=BINARY) for _ in range(N*alpha)]
z = [[model.add_var(var_type=BINARY) for _ in range(N*alpha)] for _ in range(N*alpha)]

for i in tqdm(range(N*alpha)):
    for j in range(N*alpha):
        model += z[i][j] <= u[i]
        model += z[i][j] <= u[j]
        model += z[i][j] >= u[i] + u[j] -1

with open('warm_start.npy', 'rb') as f:
    warm_start = (np.load(f)+1)/2

model_warm_start = []

for i,value in enumerate(list(warm_start.T[0])):
    model_warm_start.append((u[i],value))
    for j,value_2 in enumerate(list(warm_start.T[0])):
        model_warm_start.append((z[i][j],value*value_2))

#model.start = model_warm_start
model.objective = minimize((-1/2)*quicksum(quicksum(J[i,j]*z[i][j] for i in range(N*alpha)) for j in range(N*alpha)) + quicksum(h[i]*u[i] for i in range(N*alpha)))
model.optimize(max_seconds=15)

processing_time = time.time() - time_1

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-16


100%|██████████| 500/500 [00:06<00:00, 73.49it/s]


Set parameter TimeLimit to value 15
Set parameter NodeLimit to value 1073741824
Set parameter SolutionLimit to value 1073741824
Set parameter IntFeasTol to value 1e-06
Set parameter Method to value 3
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[rosetta2])
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 750000 rows, 250500 columns and 1749500 nonzeros
Model fingerprint: 0xcdf9d2ac
Variable types: 0 continuous, 250500 integer (250500 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e-22, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective -0.0016000
Presolve removed 1500 rows and 500 columns (presolve time = 5s) ...
Presolve removed 374040 rows and 500 columns
Presolve time: 7.65s
Presolved: 375960 rows, 250000 columns, 884414 nonzeros
Variable types: 0 continuous, 250000 integer (250000 binary)
Found heuristic solution: objective -1.

In [51]:
if model.num_solutions:
    print("Processing time: ", processing_time)
    
    spins = []
    for i in range(N*alpha):
        if u[i].x >= 0.99:
            spins.append(1)
        else:
            spins.append(-1)

    print("Spins: ", spins)
    print("Objective function: ", (-1/2)*sum([sum([test_J[i,j]*spins[j] for j in range(N*alpha)])*spins[i] for i in range(N*alpha)]) + sum([test_h[i]*spins[i] for i in range(N*alpha)])[0])

Processing time:  26.580255031585693
Spins:  [1, -1, -1, -1, -1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, 1, -1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1, -1, -1, 1, -1, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, 1, 1, -1, 1, 1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, -1, 1, 1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 1, -1, 1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, -1, 1, -1, -1, -1, 1, 1, -1, -1, -1, -